<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #8: Modelado de temas
`Fecha de entrega: 🎃 Octubre 31, 2020. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [14]:
import re
import pandas as pd 
import numpy as np
from pprint import pprint
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')
#!pip install pyLDAvis
import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [11]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving reviews_vidjew_es.csv to reviews_vidjew_es.csv
User uploaded file "reviews_vidjew_es.csv" with length 232489 bytes


In [12]:
# Cargar datos
path = 'reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [20]:

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto

data['Pre-Processed'] = data['review_body'].apply(lambda texto: pre_procesado(texto))
data.head(3)


,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,P_P,Pre-Processed
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]","[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw...","[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl...","[funciona, nintendo, switch, forma, emparejarl..."


###  `[13 pts]` Punto 2: Modelo de LDA

In [22]:
# Crear una representación de los documentos en forma de diccionario
dictionary = Dictionary(data['Pre-Processed'].values)

# Filtrar palabras muy frecuentes o infrecuentes
dictionary.filter_extremes(no_below=5, no_above=0.5)

corpus = [dictionary.doc2bow(text) for text in data['Pre-Processed'].values]

#Train the topic model
model = LdaModel(corpus=corpus, random_state=1234, id2word=dictionary, num_topics=7, passes=40)

###  `[25 pts]` Punto 3: Visualización de LDA

In [23]:
pprint(model.print_topics(num_words=10))

[(0,
  '0.031*"llegó" + 0.028*"tiempo" + 0.027*"pedido" + 0.021*"igual" + '
  '0.020*"perfecto" + 0.019*"ve" + 0.017*"artículo" + 0.016*"va" + '
  '0.016*"bien" + 0.015*"cadena"'),
 (1,
  '0.061*"bien" + 0.036*"bonito" + 0.031*"calidad" + 0.026*"precio" + '
  '0.020*"regalo" + 0.019*"parece" + 0.018*"aunque" + 0.017*"bastante" + '
  '0.014*"colgante" + 0.014*"tamaño"'),
 (2,
  '0.038*"producto" + 0.034*"pulsera" + 0.032*"llegado" + 0.031*"día" + '
  '0.023*"bonita" + 0.018*"si" + 0.017*"plata" + 0.014*"nunca" + 0.013*"q" + '
  '0.011*"regalo"'),
 (3,
  '0.084*"calidad" + 0.060*"precio" + 0.058*"buena" + 0.040*"buen" + '
  '0.025*"bueno" + 0.022*"puede" + 0.017*"entrega" + 0.015*"mala" + '
  '0.014*"sonido" + 0.014*"diseño"'),
 (4,
  '0.040*"mando" + 0.039*"si" + 0.035*"bien" + 0.024*"ps" + 0.020*"original" + '
  '0.019*"mejor" + 0.019*"juegos" + 0.018*"switch" + 0.015*"solo" + '
  '0.015*"nintendo"'),
 (5,
  '0.038*"juego" + 0.027*"mas" + 0.026*"perfecto" + 0.020*"gracias" + '
  '0.020

In [27]:
lda_display = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)